In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df = pd.read_csv('C:/Users/Home/OneDrive/Рабочий стол/dataset_MO/train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [7]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7867401104915408+-0.00852135511666111


1. Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [9]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [10]:
rf_classifier = Pipeline([
    ('features',feats),
    ('rf_classifier', RandomForestClassifier(random_state = 42)),
])

#запустим кросс-валидацию
rf_cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=16, scoring='roc_auc')
rf_cv_score = np.mean(rf_cv_scores)
rf_cv_score_std = np.std(rf_cv_scores)
print('CV score is {}+-{}'.format(rf_cv_score, rf_cv_score_std))

CV score is 0.7734501681056019+-0.007171140345435727


In [12]:
gb_classifier = Pipeline([
    ('features',feats),
    ('gb_classifier', GradientBoostingClassifier(random_state = 42)),
])

#запустим кросс-валидацию
gb_cv_scores = cross_val_score(gb_classifier, X_train, y_train, cv=16, scoring='roc_auc')
gb_cv_score = np.mean(gb_cv_scores)
gb_cv_score_std = np.std(gb_cv_scores)
print('CV score is {}+-{}'.format(gb_cv_score, gb_cv_score_std))

CV score is 0.8025124517417064+-0.007075372179995901


In [13]:
n_estimators = [50, 70]
max_depth = [3, 5, 7]

for n in n_estimators:
    for depth in max_depth:
        gb_classifier = Pipeline([
            ('features',feats),
            ('gb_classifier', GradientBoostingClassifier(n_estimators = n, max_depth = depth, random_state = 42)),
        ])

        #запустим кросс-валидацию
        gb_cv_scores = cross_val_score(gb_classifier, X_train, y_train, cv=16, scoring='roc_auc')
        gb_cv_score = np.mean(gb_cv_scores)
        gb_cv_score_std = np.std(gb_cv_scores)
        print('n_estimators: {}, max_depth = {}, CV score is {}+-{}'.format(n, depth, gb_cv_score, gb_cv_score_std))

n_estimators: 50, max_depth = 3, CV score is 0.8011686489539476+-0.006773234653669673
n_estimators: 50, max_depth = 5, CV score is 0.8026120551097841+-0.006754809478215111
n_estimators: 50, max_depth = 7, CV score is 0.8009839735576266+-0.0064752474409542765
n_estimators: 70, max_depth = 3, CV score is 0.8021323612985394+-0.006954429860504813
n_estimators: 70, max_depth = 5, CV score is 0.8026084503778794+-0.006877624100453979
n_estimators: 70, max_depth = 7, CV score is 0.8005797675929417+-0.0065032991706983345


In [14]:
gb_classifier = Pipeline([
    ('features',feats),
    ('gb_classifier', GradientBoostingClassifier(n_estimators = 70, max_depth = 5, min_samples_leaf=15, random_state = 42)),
])

#запустим кросс-валидацию
gb_cv_scores = cross_val_score(gb_classifier, X_train, y_train, cv=16, scoring='roc_auc')
gb_cv_score = np.mean(gb_cv_scores)
gb_cv_score_std = np.std(gb_cv_scores)
print('CV score is {}+-{}'.format(gb_cv_score, gb_cv_score_std))

CV score is 0.8029129182667756+-0.006701221507453466


In [15]:
def evaluate_prediction(classifier, X_train, y_train, X_test, y_test, b=1):
    classifier.fit(X_train, y_train)
    preds = classifier.predict_proba(X_test)[:, 1]
    
    #Определим метрики
    precision, recall, threshold = precision_recall_curve(y_test, preds)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    ix = np.argmax(fscore)
    
    return precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds)

In [16]:
logreg_prec, logreg_rec, logreg_fscore, logreg_roc_auc = evaluate_prediction(classifier, X_train, y_train, X_test, y_test)
rf_prec, rf_rec, rf_fscore, rf_roc_auc = evaluate_prediction(rf_classifier, X_train, y_train, X_test, y_test)
gb_prec, gb_rec, gb_fscore, gb_roc_auc = evaluate_prediction(gb_classifier, X_train, y_train, X_test, y_test)

3. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [17]:
comparing_results = pd.DataFrame({
    'classifier': ['Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    'precision': [logreg_prec, rf_prec, gb_prec],
    'recall': [logreg_rec, rf_rec, gb_rec],
    'f score': [logreg_fscore, rf_fscore, gb_fscore],
    'roc auc score': [logreg_roc_auc, rf_roc_auc, gb_roc_auc]
})

comparing_results

,classifier,precision,recall,f score,roc auc score
0,Logistic Regression,0.647431,0.837558,0.730323,0.784035
1,Random Forest,0.642669,0.815553,0.718863,0.771037
2,Gradient Boosting,0.661640,0.839171,0.739906,0.802745
